<b><center><font size='100' color=maroon>Hospital Locator</font></center></b>

![](codes.jpg)

In [5]:
import numpy as np
import pandas as pd
from math import sin, cos, sqrt, atan2, radians

In [6]:
allcity = pd.read_csv('allcity.csv',encoding="ISO-8859-1")
allcity.head()
hoscity = pd.read_csv('city2.csv',encoding="ISO-8859-1")
hoscity.head()
hospital = pd.read_csv('hos.csv')
hospital.head()

,DRG Definition,Provider Id,Provider Name,Provider Street Address,Provider City,Provider State,Provider Zip Code,Hospital Referral Region Description,Total Discharges,Average Covered Charges,Average Total Payments,Average Medicare Payments
0,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,AL - Dothan,91,$32963.07,$5777.24,$4763.73
1,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10005,MARSHALL MEDICAL CENTER SOUTH,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,AL - Birmingham,14,$15131.85,$5787.57,$4976.71
2,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10006,ELIZA COFFEE MEMORIAL HOSPITAL,205 MARENGO STREET,FLORENCE,AL,35631,AL - Birmingham,24,$37560.37,$5434.95,$4453.79
3,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10011,ST VINCENT'S EAST,50 MEDICAL PARK EAST DRIVE,BIRMINGHAM,AL,35235,AL - Birmingham,25,$13998.28,$5417.56,$4129.16
4,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10016,SHELBY BAPTIST MEDICAL CENTER,1000 FIRST STREET NORTH,ALABASTER,AL,35007,AL - Birmingham,18,$31633.27,$5658.33,$4851.44


# Function to calculate distance

In [14]:
def distance(city,rang):
    val1 = allcity.loc[allcity['city'] == city]
    lat1 = val1['lat']
    
    lng1 = val1['lng']
    lat2 = lat1.values
    lng2 = lng1.values
    
    hoscitytemp = hoscity.copy() 
    R = 6373.0
    lat = radians(lat2)
    lng = radians(lng2)
    hoscitytemp['lat'] = np.radians(hoscitytemp['lat'])
    hoscitytemp['lng'] = np.radians(hoscitytemp['lng'])

    dlon = (hoscitytemp['lat'] - lat).values
    dlat = (hoscitytemp['lng'] - lng).values
    lat3 = hoscitytemp['lat'].values
    #formula used to calculate distance between two cities(target city and patient's city) from latitude/longitude 
    a = np.sin(dlat / 2)**2 + np.cos(lat) * np.cos(lat3) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    distance = R * c
    
    hoscitytemp['distance'] = distance
    finalcity = hoscitytemp.loc[hoscitytemp['distance']<rang]
    return finalcity

# Driver Function

In [15]:
def suggest(city,dist,code):
    finalcity = distance(city,dist) # distance fn called to get eligible cities , parameter(str,float)
    hospital['code'] = hospital['DRG Definition'].astype(str).str[0]+hospital['DRG Definition'].astype(str).str[1]+hospital['DRG Definition'].astype(str).str[2]
    
    #Filtering Hospitals in the eligible cities
    hospitalincity =hospital[hospital['Provider City'].isin(finalcity['city'])]
    
    #Filtering Hospitals by required Diagonosis
    availablehospital = hospitalincity[hospitalincity['code']==code]
    
    #total expenses
    exp1 = availablehospital[' Average Covered Charges '].str[1:].values
    exp2 = availablehospital[' Average Total Payments '].str[1:].values
    exp3 = availablehospital['Average Medicare Payments'].str[1:].values
    
    avg = (exp1.astype(float)+exp2.astype(float)+exp3.astype(float))/3
    availablehospital['average'] = avg
    
    #top 10 recommended results based on minimum average expenses
    final_result =availablehospital.nsmallest(10,'average')
    final_result = final_result[['Provider Name','Provider City']]
    print(final_result)

In [19]:

#user Input
city =input("PLEASE ENTER YOUT CITY -  ")
dist =input('PLEASE ENTER THE YOUR RANGE OF HOSPITAL - ')
code = input('PLEASE ENTER DIAGONOSE CODE - ')
suggest(city,float(dist),str(code))


PLEASE ENTER YOUT CITY -  GENEVA
PLEASE ENTER THE YOUR RANGE OF HOSPITAL - 56
PLEASE ENTER DIAGONOSE CODE - 286
                                           Provider Name Provider City
58356                   ALAMANCE REGIONAL MEDICAL CENTER    BURLINGTON
58155                              LAHEY CLINIC HOSPITAL    BURLINGTON
58597                 FLETCHER ALLEN HOSPITAL OF VERMONT    BURLINGTON
58404             FORT HAMILTON HUGHES MEMORIAL HOSPITAL      HAMILTON
58297  JFK MEDICAL CTR - ANTHONY M. YELENCSICS COMMUNITY        EDISON
58298   ROBERT WOOD JOHNSON UNIVERSITY HOSPITAL HAMILTON      HAMILTON
58578                        MEDICAL CENTER OF ARLINGTON     ARLINGTON


C:\Users\Abhishek\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


Data is taken from https://www.medicare.gov/download/downloaddb.asp
